In [1]:
import pandas as pd

df = pd.read_csv("final_merged_articles.csv", sep=None, engine="python", on_bad_lines="skip")
df.info()
df.head(3)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019 entries, 0 to 2018
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_article  2019 non-null   object
 1   title       2012 non-null   object
 2   content     2019 non-null   object
 3   url         2019 non-null   object
 4   category    2019 non-null   object
 5   source      2019 non-null   object
 6   image       2019 non-null   object
dtypes: object(7)
memory usage: 110.5+ KB


,id_article,title,content,url,category,source,image
0,52633a4d3202,Apple pulls US immigration official tracking apps,A number of apps were released this year in re...,https://www.bbc.com/news/articles/c708y1egzlko,technology,BBC,https://ichef.bbci.co.uk/news/1024/branded_new...
1,c7ef2347bbf7,"TikTok recommends porn to children, says report","TikTok recommends porn to children, says repor...",https://www.bbc.com/news/articles/c708v7qkeg1o,technology,BBC,https://ichef.bbci.co.uk/news/1024/branded_new...
2,ea303e4d353c,Japan faces Asahi beer shortage after cyber-at...,"Japan is facing a shortage of Asahi products, ...",https://www.bbc.com/news/articles/c0r0y14ly5ro,technology,BBC,https://ichef.bbci.co.uk/news/1024/branded_new...


# Install libraries and Pick a pretrained encoder

In [2]:
!pip -q install sentence-transformers tldextract trafilatura tqdm

from sentence_transformers import SentenceTransformer, util
import numpy as np, re, pandas as pd
from tqdm import tqdm

# Choose encoder (English)
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(MODEL_NAME)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 23.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Load your dataset

In [3]:

df = pd.read_csv("final_merged_articles.csv", sep=None, engine="python", on_bad_lines="skip")
df.info()
df.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019 entries, 0 to 2018
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_article  2019 non-null   object
 1   title       2012 non-null   object
 2   content     2019 non-null   object
 3   url         2019 non-null   object
 4   category    2019 non-null   object
 5   source      2019 non-null   object
 6   image       2019 non-null   object
dtypes: object(7)
memory usage: 110.5+ KB


,id_article,title,content,url,category,source,image
0,52633a4d3202,Apple pulls US immigration official tracking apps,A number of apps were released this year in re...,https://www.bbc.com/news/articles/c708y1egzlko,technology,BBC,https://ichef.bbci.co.uk/news/1024/branded_new...
1,c7ef2347bbf7,"TikTok recommends porn to children, says report","TikTok recommends porn to children, says repor...",https://www.bbc.com/news/articles/c708v7qkeg1o,technology,BBC,https://ichef.bbci.co.uk/news/1024/branded_new...
2,ea303e4d353c,Japan faces Asahi beer shortage after cyber-at...,"Japan is facing a shortage of Asahi products, ...",https://www.bbc.com/news/articles/c0r0y14ly5ro,technology,BBC,https://ichef.bbci.co.uk/news/1024/branded_new...


***Uppercase to lowercase***

In [4]:
df["source"] = df["source"].str.lower()


# Build a domain text bundle

In [5]:
import re

def clean_text(x):
    if pd.isna(x): return ""
    x = re.sub(r"\s+", " ", str(x)).strip()
    return x

# Group by source (for now: one group "bbc")
grouped = df.assign(title=df["title"].map(clean_text),
                    content=df["content"].map(clean_text)) \
            .groupby("source")

domain_texts = {}

# Limit to N docs to keep it fast; you can raise N later
N = 500
for domain, sub in grouped:
    sample = sub.sample(n=min(N, len(sub)), random_state=42)
    # Build a small corpus: “title. content”
    texts = (sample["title"].fillna("") + ". " + sample["content"].fillna("")).tolist()
    # Optional: trim very long items
    texts = [t[:4000] for t in texts if len(t) > 50]
    domain_texts[domain] = texts

list(domain_texts.keys()), len(domain_texts.get("bbc", []))


(['aljazeera',
  'arstechnica',
  'bbc',
  'cbssports',
  'cnn',
  'livescience',
  'new york times',
  'the guardian',
  'theconversation',
  'variety',
  'wired'],
 500)

# Encode and pool to one vector per domain

In [8]:
import torch
from tqdm import tqdm

def encode_corpus(texts, batch_size=16):
    if not texts: return None
    emb = model.encode(texts, batch_size=batch_size, convert_to_tensor=True, show_progress_bar=True)
    # Mean pool across documents
    v = emb.mean(dim=0)
    return v / v.norm()

domain_vectors = {}
for domain, texts in tqdm(domain_texts.items()):
    v = encode_corpus(texts)
    domain_vectors[domain] = v

domain_vectors.keys()


  0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

  9%|▉         | 1/11 [00:26<04:29, 26.97s/it]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

 18%|█▊        | 2/11 [00:52<03:54, 26.09s/it]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 91%|█████████ | 10/11 [35:40<03:14, 194.85s/it]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 11/11 [36:46<00:00, 200.63s/it]


dict_keys(['aljazeera', 'arstechnica', 'bbc', 'cbssports', 'cnn', 'livescience', 'new york times', 'the guardian', 'theconversation', 'variety', 'wired'])

# Build a trusted centroid μ_trusted

In [9]:
import trafilatura, requests
from urllib.parse import urljoin

HEADERS = {"User-Agent": "Mozilla/5.0 (Colab Reliability Bot)"}
ABOUT_CANDIDATES = ["about","about-us","who-we-are","a-propos","aboutus"]
POLICY_CANDIDATES = ["corrections","editorial","ethics","standards","policy","policies"]

trusted_domains = [
    "apnews.com","reuters.com","ft.com","wsj.com","npr.org","bbc.com",
    "nytimes.com","economist.com","nature.com","sciencemag.org"
]

def fetch(url, timeout=10):
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout, allow_redirects=True)
        if r.status_code == 200 and r.text:
            return r.text
    except Exception:
        pass
    return None

def extract_text(html):
    return trafilatura.extract(html, include_comments=False, include_tables=False)

def find_pages(domain):
    texts = []
    # Try https/http home
    for home in [f"https://{domain}/", f"http://{domain}/"]:
        h = fetch(home)
        if h:
            t = extract_text(h)
            if t and len(t) > 600: texts.append(("home", t[:6000]))
            # try about/policy slugs
            for slug in ABOUT_CANDIDATES:
                html = fetch(urljoin(home, slug))
                if html:
                    tt = extract_text(html)
                    if tt and len(tt) > 400: texts.append(("about", tt[:6000])); break
            for slug in POLICY_CANDIDATES:
                html = fetch(urljoin(home, slug))
                if html:
                    tt = extract_text(html)
                    if tt and len(tt) > 400: texts.append(("policy", tt[:6000])); break
            break
    return texts

trusted_vecs = []
for d in tqdm(trusted_domains):
    pages = find_pages(d)
    pages_texts = [txt for _, txt in pages]
    if pages_texts:
        v = encode_corpus(pages_texts, batch_size=8)
        if v is not None:
            trusted_vecs.append(v)

if not trusted_vecs:
    raise RuntimeError("Failed to build μ_trusted (no pages fetched). Add more trusted domains or use your pre-collected texts.")

mu_trusted = torch.stack(trusted_vecs, dim=0).mean(dim=0)
mu_trusted = mu_trusted / mu_trusted.norm()


  0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 20%|██        | 2/10 [00:08<00:27,  3.47s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 50%|█████     | 5/10 [00:34<00:46,  9.27s/it]

...

 60%|██████    | 6/10 [00:41<00:32,  8.21s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 80%|████████  | 8/10 [00:47<00:10,  5.19s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:59<00:00,  5.91s/it]


# Score your (BBC) domain

In [10]:
def cosine01(a, b):
    # expects L2-normalized tensors
    return float((torch.dot(a, b).item() + 1) / 2)

scores = []
for domain, v in domain_vectors.items():
    s = cosine01(v, mu_trusted)
    scores.append({"domain": domain, "score": round(s, 3)})

scores


[{'domain': 'aljazeera', 'score': 0.718},
 {'domain': 'arstechnica', 'score': 0.685},
 {'domain': 'bbc', 'score': 0.759},
 {'domain': 'cbssports', 'score': 0.625},
 {'domain': 'cnn', 'score': 0.831},
 {'domain': 'livescience', 'score': 0.662},
 {'domain': 'new york times', 'score': 0.878},
 {'domain': 'the guardian', 'score': 0.857},
 {'domain': 'theconversation', 'score': 0.831},
 {'domain': 'variety', 'score': 0.732},
 {'domain': 'wired', 'score': 0.694}]

# Turn score into a label

In [11]:
def label_from_score(s):
    if s >= 0.85: return "Reliable"
    if s >= 0.70: return "Mostly reliable"
    if s >= 0.55: return "Mixed"
    if s >= 0.40: return "Questionable"
    return "Unreliable"

for item in scores:
    item["label"] = label_from_score(item["score"])

import pandas as pd
out = pd.DataFrame(scores).sort_values("score", ascending=False)
out


,domain,score,label
6,new york times,0.878,Reliable
7,the guardian,0.857,Reliable
8,theconversation,0.831,Mostly reliable
4,cnn,0.831,Mostly reliable
2,bbc,0.759,Mostly reliable
9,variety,0.732,Mostly reliable
0,aljazeera,0.718,Mostly reliable
10,wired,0.694,Mixed
1,arstechnica,0.685,Mixed
5,livescience,0.662,Mixed


***Save results***

In [13]:
out.to_csv("source_reliability_scores.csv", index=False)
from google.colab import files
files.download("source_reliability_scores.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Install LangGraph

In [14]:
!pip -q install langgraph langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


***Define the shared state (used between nodes)***

In [15]:
from typing import TypedDict, List, Optional

class AgentState(TypedDict, total=False):
    # inputs
    query: Optional[str]
    source: Optional[str]
    # your outputs
    source_score: Optional[float]
    source_label: Optional[str]
    source_snippets: Optional[List[str]]
    # general notes/logs
    notes: Optional[List[str]]


# Define your Source Reliability Node

In [17]:
out.columns


Index(['domain', 'score', 'label'], dtype='object')

In [18]:
import pandas as pd

# Make lookup table (faster)
_table = out.set_index("domain")

def source_reliability_node(state: AgentState) -> AgentState:
    d = (state.get("domain") or "").strip().lower()

    # Loose match if user gives variants
    if d not in _table.index:
        for cand in _table.index:
            if d in cand or cand in d:
                d = cand
                break

    if d not in _table.index:
        notes = state.get("notes", [])
        notes.append(f"Domain '{state.get('domain')}' not found in reliability table.")
        return {**state, "notes": notes}

    row = _table.loc[d].to_dict()
    return {
        **state,
        "domain": d,
        "source_score": float(row["score"]),
        "source_label": row["label"],
        "source_snippets": (row.get("snippets") or [])[:3] if "snippets" in row else [],
    }



# Build and compile the LangGraph

In [22]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(AgentState)
graph.add_node("source_reliability", source_reliability_node)
graph.set_entry_point("source_reliability")
graph.add_edge("source_reliability", END)

app = graph.compile(checkpointer=MemorySaver())


**test**

In [23]:
final_state = app.invoke(
    {"domain": "bbc"},
    config={"configurable": {"thread_id": "sr-demo-1"}}  # any string id
)
final_state



{'source_score': 0.878, 'source_label': 'Reliable', 'source_snippets': []}